CNN

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 6.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=d270e8a8c0c12da9e6062d4dbf8b4ace5c74d78bc4ad5e92d02c4b412fae6df0
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=002221979d6e7831ebfa217f080ec8941b984c0b6ff6eb79272db3b07fd45852
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [6]:
print(tf.__version__)

2.4.1


In [7]:
fashion_mnist=keras.datasets.fashion_mnist

In [9]:
(train_images,train_labels),(test_images,test_lebels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [10]:
train_images=train_images/255
test_images=test_images/255

In [11]:
test_images.shape

(10000, 28, 28)

In [16]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [17]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [18]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [19]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [20]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 19s]
val_accuracy: 0.8608333468437195

Best val_accuracy So Far: 0.9154999852180481
Total elapsed time: 00h 01m 45s
INFO:tensorflow:Oracle triggered exit


In [21]:
model=tuner_search.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,676,538
Trainable params: 2,676,538
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1179 - accuracy: 0.9570 - val_loss: 0.2430 - val_accuracy: 0.9178
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0766 - accuracy: 0.9734 - val_loss: 0.3163 - val_accuracy: 0.9128
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0530 - accuracy: 0.9818 - val_loss: 0.3163 - val_accuracy: 0.9175
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0345 - accuracy: 0.9871 - val_loss: 0.3754 - val_accuracy: 0.9110
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0270 - accuracy: 0.9907 - val_loss: 0.4040 - val_accuracy: 0.9162
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0193 - accuracy: 0.9938 - val_loss: 0.4695 - val_accuracy: 0.9122
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0185 - accuracy: 0.9934 - val_loss: 0.5272 - val_accuracy